In [1]:
!pip install allennlp==2.1.0 allennlp-models==2.1.0
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install stanza

     |████████████████████████████████| 585 kB 6.0 MB/s 
     |████████████████████████████████| 407 kB 60.9 MB/s 
     |████████████████████████████████| 132 kB 74.9 MB/s 
     |████████████████████████████████| 125 kB 56.8 MB/s 
     |████████████████████████████████| 593 kB 69.3 MB/s 
     |████████████████████████████████| 12.8 MB 64.8 MB/s 
     |████████████████████████████████| 1.3 MB 63.4 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
     |████████████████████████████████| 1.9 MB 58.0 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 9.9 MB 44.6 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 140 kB 77.3 MB/s 
     |████████████████████████████████| 127 kB 60.0 MB/s 
     |████████████████████████████████| 880 kB 69.0 MB/s 
     |████████████████████████████████| 3.3 MB 59.5 MB/s 
  Created wheel for 

In [2]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import stanza
stanza.install_corenlp()
from stanza.server import CoreNLPClient

2022-11-11 02:54:05 INFO: Installing CoreNLP package into /root/stanza_corenlp


In [3]:
import math
import re
from collections import Counter
import spacy
import json
import os

WORD = re.compile(r"\w+")
coref = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Plugin allennlp_models could not be loaded: No module named 'nltk.translate.meteor_score'


[nltk_data] Downloading package wordnet to /root/nltk_data...


downloading: 100%|##########| 1345986155/1345986155 [00:27<00:00, 48072222.28B/s]


Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# from google.colab import drive
# drive.mount('/content/GoogleDrive/')
# data_dir = '/content/GoogleDrive/MyDrive/results/meta-transcript-readable-pymupdf.txt'
!unzip cleaned_json.zip
data_dir = '/content/cleaned_json'

Archive:  cleaned_json.zip
   creating: cleaned_json/
  inflating: cleaned_json/1803.05663.json  
  inflating: cleaned_json/2010.12415.json  
  inflating: cleaned_json/2204.13102.json  
  inflating: cleaned_json/2205.06338.json  
  inflating: cleaned_json/2102.07425.json  
  inflating: cleaned_json/2010.15403.json  
  inflating: cleaned_json/2010.01337.json  
  inflating: cleaned_json/2109.12142.json  
  inflating: cleaned_json/1912.11166.json  
  inflating: cleaned_json/1912.03311.json  
  inflating: cleaned_json/2008.09667.json  
  inflating: cleaned_json/2109.15052.json  
  inflating: cleaned_json/2112.15036.json  
  inflating: cleaned_json/2011.02612.json  
  inflating: cleaned_json/2201.02729.json  
  inflating: cleaned_json/1903.00472.json  
  inflating: cleaned_json/2204.12929.json  
  inflating: cleaned_json/1908.02591.json  
  inflating: cleaned_json/1805.08550.json  
  inflating: cleaned_json/2201.12893.json  
  inflating: cleaned_json/2201.05906.json  
  inflating: cleaned_j

In [5]:
# Cosine similarity between two vectors
def get_cosine(vec1, vec2):
  intersection = set(vec1.keys()) & set(vec2.keys())
  numerator = sum([vec1[x] * vec2[x] for x in intersection])

  sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
  sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
  denominator = math.sqrt(sum1) * math.sqrt(sum2)

  if not denominator:
      return 0.0
  else:
      return float(numerator) / denominator

# Convert string to a vector
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

# check if 2 triples exist with similar subject, object, predicate longer subject is taken
def subset_phrase(triples, simScore):
  n = len(triples)
  new_triple = triples[:]
  for i in range(n):
    firstTri = triples[i]
    for j in range(i + 1, n):
      secondTri = triples[j]
      text1 = firstTri[0] + " " + firstTri[1] + " " + firstTri[2] + " " + firstTri[3]
      text2 = secondTri[0] + " " + secondTri[1] + " " + secondTri[2] + " " + secondTri[3]
      vector1 = text_to_vector(text1)
      vector2 = text_to_vector(text2)
      # Doing the above eliminates worrying about scenarios of exactly the same subject, object or predicate
      if get_cosine(vector1, vector2) >= simScore:
        # temp = firstTri if len(firstTri[0]) > len(secondTri[0]) else secondTri #can make this based on the subject, not text
        temp = firstTri if len(text1) > len(text2) else secondTri
        if temp == secondTri:
          if firstTri in new_triple:
            new_triple.remove(firstTri)
        elif secondTri in new_triple:
            new_triple.remove(secondTri)
  return new_triple

# return true or false if triple is a valid triple
def filter_triple(triple):
  DAY_OF_THE_WEEK = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
  subject = triple[0]
  predicate = triple[1]
  object_ = triple[2]
  doc_sub = nlp(subject)
  doc_obj = nlp(object_)
  doc_pred = nlp(predicate)
  subject_pos = [token.pos_ for token in doc_sub] #all parts of speech
  object_pos = [token.pos_ for token in doc_obj] #all parts of speech
  predicate_pos = [token.pos_ for token in doc_pred] #all parts of speech
  all_words_day_week = [True if word.lower() in DAY_OF_THE_WEEK else False for word in subject.split()]
  contains_day_of_week = any(all_words_day_week) #performs OR operation of booleans in list
  if 'NOUN' not in subject_pos and 'PROPN' not in subject_pos and contains_day_of_week:
    return False
  elif ('VERB' in subject_pos) or ('VERB' in object_pos):
    return False
  elif any(["PRON" in subject_pos,"PRON" in predicate_pos, "PRON" in object_pos]):
    return False
  elif subject == object_:
    return False
  return True

In [6]:
def addTriple(document, simThresh):
  allTriples = []
  triples = []
  for sentence in document['sentences']:
    # Add temporal relation
    cursentTemp = ""
    for ele in sentence['entitymentions']:
      if ele['ner'] == "DATE":
        temp = ele['text'] 
        cursentTemp = temp if len(temp) > len(cursentTemp) else cursentTemp    
    sentTrip = []
    for triple in sentence['openie']:
      if filter_triple((triple['subject'], triple['relation'], triple['object'])):
        sentTrip.append((triple['subject'], triple['relation'], triple['object'], cursentTemp))
    valid_triples = subset_phrase(sentTrip, simThresh) #more similarity score increases the number of sentences retrieved since we're making sure sentences are extremely close in order to drop the shorter one. 
    triples.append(valid_triples)
    
  triples = [item for sublist in triples for item in sublist]
  for ele in triples:
    tmp_arg = ele[3]
    if len(tmp_arg) > 0:
      allTriples.append((ele[2], ele[1], ele[3]))
    allTriples.append((ele[0], ele[1], ele[2]))
  return allTriples

In [7]:
# PREPROCESS TEXT
def clean_text(sentence):
  """
  Input sentence: Raw sentence
  Output sentence: cleaned sentence with - 
                  (i) no extra whitespaces, no new lines, no tabs
                  (ii) lemmatized sentence
                  (iii) generic sentences with no NER are returned as empty string

  """
  doc=nlp(sentence)
  # lemmatization
  lemmatized_sentence=""
  for token in doc:
    if token.lemma_ !="-PRON-":
      lemmatized_sentence=lemmatized_sentence+token.lemma_+" "
    else:
      if token.lemma_ == " . ":
        print("MAD OO")
      lemmatized_sentence=lemmatized_sentence+str(token)+" "
  sentence=lemmatized_sentence[:-1]

  # removing whitespace, /n, tabs
  sentence = sentence.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
  pattern = re.compile(r'\s+') 
  Without_whitespace = re.sub(pattern, ' ', sentence)
  # There are some instances where there is no space after '?' & ')', 
  # So I am replacing these with one space so that It will not consider two words as one token.
  sentence = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
  return sentence


text = "Ridwan is not a person. He is nice."
print(clean_text(text))

Ridwan be not a person . He be nice .


In [8]:
# Load files from source document
allTriples = []
files = sorted(os.listdir(data_dir))
# allText = ""
for file in files:
  with open(f'{data_dir}/{file}') as f:
    d = json.load(f)
  text = d['text']
  all_authors = ""
  for ele in d['auhtors']:
    all_authors += ele + ", "
  allTriples.append((all_authors[:-2], 'author(s)', d['title']))
  if len(text) > 0:
    text = clean_text(text)
    coref_text = coref.coref_resolved(document=text)
    with CoreNLPClient(timeout=150000000, be_quiet=False, annotators=['ner', 'openie'], memory='25G', endpoint='http://localhost:9001') as client:
      document = client.annotate(coref_text, output_format='json', properties={"openie.triple.all_nominals": True})
      allTriples.extend(addTriple(document, 0.5))
    # allText += coref_text + " "

2022-11-11 02:55:48 INFO: Writing properties to tmp file: corenlp_server-08bd815a86e24245.props
2022-11-11 02:55:48 INFO: Starting server with command: java -Xmx25G -cp /root/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-08bd815a86e24245.props -annotators ner,openie -preload -outputFormat serialized
2022-11-11 02:56:44 INFO: Writing properties to tmp file: corenlp_server-14f8c14dc3e94695.props
2022-11-11 02:56:44 INFO: Starting server with command: java -Xmx25G -cp /root/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-14f8c14dc3e94695.props -annotators ner,openie -preload -outputFormat serialized
2022-11-11 02:57:32 INFO: Writing properties to tmp file: corenlp_server-4abd927eed2d41dd.props
2022-11-11 02:57:32 INFO: Starting server with

In [9]:
# allText = """An interest rate is the rental price of money. The concepts of supply, demand and equilibrium apply in this market just as they do in other markets. This market is referred to as the market for loanable funds. In the market for loanable funds, the suppliers of funds are economic entities that currently have a surplus in their budget. In other words, they have more income than they currently want to spend; they would like to save some of their money and spend it in future time periods. Instead of just putting these savings in a box on a shelf for safekeeping until they want to spend it, they can let someone else borrow that money. In essence, they are renting that money to someone else, who pays a rental price called the interest rate."""
# allText = coref.coref_resolved(document=allText)
# allTriples = []

# with CoreNLPClient(timeout=150000000, be_quiet=False, annotators=['ner', 'openie'], memory='25G', endpoint='http://localhost:9001') as client:
#   document = client.annotate(allText, output_format='json', properties={"openie.triple.all_nominals": True})
#   allTriples.extend(addTriple(document, 0.5))


In [10]:
dr = {"results" : allTriples}
json_object = json.dumps(dr)
with open("stanfordopenie.json", "w") as f:
  f.write(json_object)